In [6]:
# 显示cell运行时长
%load_ext klab-autotime

In [1]:
import pandas as pd
from sklearn import model_selection
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
import time
import matplotlib.pyplot as plt
import seaborn as sns
import gc

In [3]:
def rate_fun(data):
    data['type3_rate'] = data['act_type3_count']/(data['act_day_count']+0.00001)
    data['type2_rate'] = data['act_type2_count']/(data['act_day_count']+0.00001)
    data['type1_rate'] = data['act_type1_count']/(data['act_day_count']+0.00001)
    data['type0_rate'] = data['act_type0_count']/(data['act_day_count']+0.00001)
    
    data['register_day'] = data['register_day'] + 1
    
    data['page0_rate'] = data['act_page0_count']/(data['act_day_count']+0.00001)
    data['page1_rate'] = data['act_page1_count']/(data['act_day_count']+0.00001)
    data['page2_rate'] = data['act_page2_count']/(data['act_day_count']+0.00001)
    data['page3_rate'] = data['act_page3_count']/(data['act_day_count']+0.00001)
    data['page4_rate'] = data['act_page4_count']/(data['act_day_count']+0.00001)
    
    data['dev1'] = data.groupby('device_type')['device_type'].transform('count').astype(np.uint16)
    
    data['device_exp_reg'] = data['device_type']/(data['register_type']+0.00001)
    
    data['device_exp_app'] = data['device_type']/(data['app_day_max']+0.00001)
    
    data['app_max_exp_std'] = data['app_day_max']/(data['app_day_std']+0.00001)
    data['app_max_add_std'] = data['app_day_max']+data['app_day_std']
    
    data['act_13_14_count'] = data['act_day13_count']+data['act_day14_count']
    
    data['app_max_mul_std'] = data['app_day_max']*data['app_day_std']
    data['app_act_mul_skew'] = data['app_day_skew']*data['act_day_skew']
    
    data['act_day15_rate'] = data['act_day15_count']/(data['act_day_count']+0.00001)
    data['act_day14_rate'] = data['act_day14_count']/(data['act_day_count']+0.00001)
    data['act_day13_rate'] = data['act_day13_count']/(data['act_day_count']+0.00001)
    
    data['type0_day15_rate'] = data['act_type0_count']/(data['act_day15_count']+0.00001)
    data['page0_day15_rate'] = data['act_page0_count']/(data['act_day15_count']+0.00001)
    
    data['type1_day15_rate'] = data['act_type1_count']/(data['act_day15_count']+0.00001)
    data['page1_day15_rate'] = data['act_page1_count']/(data['act_day15_count']+0.00001)
    
    data['type2_day15_rate'] = data['act_type2_count']/(data['act_day15_count']+0.00001)
    data['page2_day15_rate'] = data['act_page2_count']/(data['act_day15_count']+0.00001)
    
    data['is_have_act_video'] = data['act_day_count']+data['video_day_count']
    act_video_mean = data['is_have_act_video'].mean()
    data['is_have_act_video'] = data['is_have_act_video'].fillna(0).apply(lambda x:0 if x<=0 else (1 if x < act_video_mean else 2))
    
    data['is_have_type012'] = data['act_type1_count']+data['act_type0_count']+data['act_type2_count']
    type012_mean = data['is_have_type012'].mean()
    data['is_have_type012_1'] = data['is_have_type012'].fillna(0).apply(lambda x:0 if x<=0 else (1 if x < type012_mean else 2))
    
    data['is_have_type12'] = data['act_type1_count']+data['act_type2_count']
    type12_mean = data['is_have_type12'].mean()
    data['is_have_type12_1'] = data['is_have_type12'].fillna(0).apply(lambda x:0 if x<=0 else (1 if x < type12_mean else 2))
    
    data['is_have_act_type12'] = data['act_type1_count']+data['act_type2_count']+data['act_day_count']
    have_act_type12_mean = data['is_have_act_type12'].mean()
    data['is_have_act_type12_1'] = data['is_have_act_type12'].fillna(0).apply(lambda x:0 if x<=0 else (1 if x < have_act_type12_mean else 2))
    return data

time: 10.4 ms


In [45]:
def train_data(train,test):

    # print(train.columns.values.tolist())
    # print(len(train.columns.values.tolist()))
    
    test_feature = test.drop("user_id",1)
    
    print("----->")
    l = pd.DataFrame()
    l['label'] = train['label']
    l = l.groupby('label',as_index=False)['label'].agg({'label_count':'count'})
    print(l)
    
    label = train['label']      
    train.drop(['user_id','label'], inplace=True, axis=1)
    
    used_feature =[
        'register_day', 'register_type', 'device_type', 'act_day_count', 'act_day_skew', 'act_day_std',
        'act_day_mean', 'act_day_max', 'act_continue_mean', 'act_day11_count', 'act_day10_count', 'act_day13_count',
        'act_day4_count', 'act_day3_count', 'act_day12_count', 'act_day8_count', 'act_day5_count', 'act_day7_count',
        'act_day9_count', 'act_day6_count', 'act_day15_count', 'act_day2_count', 'act_day14_count', 'act_page0_count', 
        'act_page2_count', 'act_page3_count', 'act_page1_count', 'act_page4_count', 'act_type0_count', 'act_type1_count', 
        'act_type3_count', 'act_type2_count', 'act_author_std', 
        'act_author_mean', 'act_author_count', 'act_last_gap', 'act_daily_mean', 'act_daily_std', 'act_video_nums', 
        'single_video_mean', 'single_video_std', 'every_video_type_std', 'every_video_type_mean', 
        'video_day_std', 'video_day_mean', 'video_day_count', 'video_day_max', 'video_day_skew', 
        'video_daily_mean', 'video_daily_std', 'app_day_count', 'app_day_mean', 'app_day_std', 
        'app_day_skew', 'app_day_max', 'app_day_diff_max', 'app_day_diff_mean', 'app_day_diff_std',
        'app_continue_mean', 'app_last_gap',  
        'device_exp_reg','type0_rate','type1_rate','type2_rate','page0_rate','page2_rate',
        'device_type_register_rate',
        'is_have_act_video','is_have_type012','is_have_type12','is_have_act_type12','is_have_act_type12_1',
        'is_have_type12_1','is_have_type012_1','dev2'
    ]
    t = train[used_feature]
    text_use_feature = test[used_feature]
    
    N = 5
    skf = StratifiedKFold(n_splits=N,shuffle=True,random_state=6666)
    
    xx_cv = []
    xx_pre = []
    train_value = t.values
    label = label.values
    test_feature = text_use_feature.values
    
    train_features = t.columns.tolist()
    df = pd.DataFrame(train_features, columns=['feature'])
    df1 = pd.DataFrame(train_features, columns=['feature'])
    i = 0
    for train_in,test_in in skf.split(train_value,label):
        print("折数",i+1)
        X_train,X_test,y_train,y_test = train_value[train_in],train_value[test_in],label[train_in],label[test_in]
        lgb_train = lgb.Dataset(X_train, y_train)
        lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)
    
        params = {
            'boosting_type': 'gbdt',
            'objective': 'binary',
            'metric': {'auc'},
            'num_leaves': 13,
            'max_depth': 4,
            'min_data_in_leaf': 300,
            'learning_rate': 0.1,
            'feature_fraction': 0.8,
            'bagging_fraction': 0.85,
            'bagging_freq': 5,
            'lambda_l1': 0.6,  
            'lambda_l2': 0.7,  # 越小l2正则程度越高
            'min_gain_to_split': 0.2,
            'verbose': 5,
            'max_bin': 90
        }
        print('开始训练......')
        gbm = lgb.train(
           params,
           lgb_train,
           num_boost_round=40000,
           valid_sets=lgb_eval,
           early_stopping_rounds=300,
           verbose_eval=300
       )
    
    # gbm.save_model('/Users/maomao/Desktop/bulesky/data/train_test/lgb_model.txt')
        y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
        xx_cv.append(roc_auc_score(y_test,y_pred))
        xx_pre.append(gbm.predict(test_feature, num_iteration=gbm.best_iteration))
        df['importance'+str(i)]=list(gbm.feature_importance())  
        i = i+1
        df1['importance'] = df.mean(1)    
        df1 = df1.sort_values(by='importance',ascending=False)    
    print (df1)
    df1.to_csv('/home/kesci/bigdata/result/feature_importance.csv',index=False)
    print (list(df1['feature']))
    
    print('xx_cv',np.mean(xx_cv))
    s = 0
    for i in xx_pre:
        s = s + i
    s = s/N
    test_label = test['label']
    print("线下验证分数",roc_auc_score(test_label,s))
    df_result = pd.DataFrame()
    df_result['user_id'] = test['user_id']
    df_result['probability'] = list(s)
    time_date = time.strftime('%Y-%m-%d',time.localtime(time.time()))
    df_result.to_csv('/home/kesci/bigdata/result/lgb_%s_%s.csv'%(str(time_date),str(np.mean(xx_cv)).split('.')[1]),sep=',',index=False,header=None)

time: 26.1 ms


In [46]:
n1 = pd.read_csv("/home/kesci/input/home/kesci/fuck/nn1.csv")
n1 = rate_fun(n1)

n2 = pd.read_csv("/home/kesci/input/home/kesci/fuck/nn2.csv")
n2 = rate_fun(n2)

n = pd.concat([n1,n2],axis=0,ignore_index=True,join='outer')

n3 = pd.read_csv("/home/kesci/input/home/kesci/fuck/nn3.csv")
n3 = rate_fun(n3)
train_data(n1,n2)

----->
   label  label_count
0      0       127952
1      1       136540
折数 1
开始训练......
Training until validation scores don't improve for 300 rounds.
[300]	valid_0's auc: 0.884299
[600]	valid_0's auc: 0.884093
Early stopping, best iteration is:
[335]	valid_0's auc: 0.884337
折数 2
开始训练......
Training until validation scores don't improve for 300 rounds.
[300]	valid_0's auc: 0.881991
[600]	valid_0's auc: 0.881899
Early stopping, best iteration is:
[347]	valid_0's auc: 0.882021
折数 3
开始训练......
Training until validation scores don't improve for 300 rounds.
[300]	valid_0's auc: 0.882517
[600]	valid_0's auc: 0.882474
Early stopping, best iteration is:
[410]	valid_0's auc: 0.88261
折数 4
开始训练......
Training until validation scores don't improve for 300 rounds.
[300]	valid_0's auc: 0.881612
Early stopping, best iteration is:
[294]	valid_0's auc: 0.88162
折数 5
开始训练......
Training until validation scores don't improve for 300 rounds.
[300]	valid_0's auc: 0.880246
[600]	valid_0's auc: 0.88023
Early

In [45]:
f1 = pd.read_csv("/home/kesci/bigdata/train1/fuck1_feature.csv")
n1 = pd.read_csv("/home/kesci/input/home/kesci/bigdata/train1/nn1_feature.csv")
n_col = n1.columns.values.tolist()
f_col = f1.columns.values.tolist()
df = []
for i in n_col:
    if i in f_col:
        df.append(i)
print(df)

['user_id', 'register_day', 'register_type', 'device_type', 'act_video_nums']
time: 2.42 s


In [81]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dropout
from keras.layers import Dense, Activation
from keras.utils.np_utils import to_categorical
from keras.callbacks import Callback
from keras import backend as K

from sklearn.preprocessing import StandardScaler # 用于特征的标准化
from sklearn.preprocessing import Imputer
from keras.layers.advanced_activations import PReLU
from keras.layers import *
from keras.callbacks import EarlyStopping,ModelCheckpoint

time: 2.26 ms


In [246]:
def get_rate(data):
    data['type0_rate'] = data['act_type0_count']/(data['act_day_count']+0.00001)
    data['type1_rate'] = data['act_type1_count']/(data['act_day_count']+0.00001)
    data['type2_rate'] = data['act_type2_count']/(data['act_day_count']+0.00001)
    data['type3_rate'] = data['act_type3_count']/(data['act_day_count']+0.00001)
    
    data['page0_rate'] = data['act_page0_count']/(data['act_day_count']+0.00001)
    data['page1_rate'] = data['act_page1_count']/(data['act_day_count']+0.00001)
    data['page2_rate'] = data['act_page2_count']/(data['act_day_count']+0.00001)
    data['page3_rate'] = data['act_page3_count']/(data['act_day_count']+0.00001)
    data['page4_rate'] = data['act_page4_count']/(data['act_day_count']+0.00001)
    
    act_mean = data['act_day_count'].mean()
    data['is_have_act'] = data['act_day_count'].apply(lambda x:0 if x<=0 else (1 if x < act_mean else 2))
    video_mean = data['video_day_count'].mean()
    data['is_have_video'] = data['video_day_count'].apply(lambda x:0 if x<=0 else (1 if x < video_mean else 2))
    type0_mean = data['act_type0_count'].mean()
    data['is_have_type0'] = data['act_type0_count'].fillna(0).apply(lambda x:0 if x<=0 else (1 if x < type0_mean else 2))
    type1_mean = data['act_type1_count'].mean()
    data['is_have_type1'] = data['act_type1_count'].fillna(0).apply(lambda x:0 if x<=0 else (1 if x < type1_mean else 2))
    type2_mean = data['act_type2_count'].mean()
    data['is_have_type2'] = data['act_type2_count'].fillna(0).apply(lambda x:0 if x<=0 else (1 if x < type2_mean else 2))
    
    act_mean = data['act_day_count'].mean()
    data['act_day_count1'] = data['act_day_count'].fillna(0).apply(lambda x:0 if x<=0 else (1 if x < act_mean else 2))
    
    act_max = data['act_day_max'].mean()
    data['act_day_max1'] = data['act_day_max'].fillna(0).apply(lambda x:0 if x<=0 else (1 if x < act_max else 2))
    data['act_day_max2'] = data['act_day_max'].fillna(0).apply(lambda x:0 if x<=0 else 1)
    page2_mean = data['act_page2_count'].mean()
    data['is_have_page2'] = data['act_page2_count'].fillna(0).apply(lambda x:0 if x<=0 else (1 if x < page2_mean else 2))
    data['every_video_rate'] = data['act_day_count']/(data['act_video_nums']+0.00001)
    data['every_video_author_rate'] = data['act_author_counts']/(data['act_video_nums']+0.00001)
    # data['author_act_rate'] =data['act_author_counts']/data['act_day_count']
    
    data['act_daily_max_rate'] = data['act_daily_max']/(data['act_day_count']+0.00001)
    data['act_single_video_rate'] = data['single_video_max']/(data['act_day_count']+0.00001)
    # data['app_daily_max_rate'] = data['app_daily_max']/(data['app_day_count']+0.00001)
    
    # data['every_page_type_rate'] = data['every_page_type_max']/data['act_daily_max']
    # data['every_video_type_rate'] = data['every_video_type_max']/data['act_daily_max']
    
    data['app_act_rate'] = data['app_day_std']/(data['act_last1days_counts']+0.00001)
    data['app_video_type_rate'] = data['app_day_max']/(data['every_video_type_mean']+0.00001)
    data['app_video_type_sub'] = data['app_day_max']-data['every_video_type_mean']
    
    data['page1_2_rate'] = data['page2_rate']/(data['page1_rate']+0.00001)
    data['app_std_mul'] = data['app_day_std']*data['page1_rate']
    data['act_last1_mul'] = data['act_last1days_counts']*data['page1_rate']
    data['act_last1_std_mul'] = (data['act_last1days_counts']+data['app_day_std'])*data['page1_rate']
    
    data['app_diff_rate'] = data['app_day_diff_std']/(data['app_day_std']+0.00001)
    data['app_page1_rate'] = data['app_day_diff_std']*data['page1_rate']
    data['page1_type2_rate'] = data['page1_rate']*data['type2_rate']
    
    data['app_std_div_mean'] = data['app_day_std']/(data['app_day_mean']+0.00001)
    data['act_std_div_mean'] = data['act_day_std']/(data['act_day_mean']+0.00001)
    data['video_std_div_mean'] = data['video_day_std']/(data['video_day_mean']+0.00001)
    
    data['register_day'] = data['register_day'] + 1
    # data['register_day'] = data['register_day'].apply(lambda x:16 if x>16 else x)
    
    data['device_type1'] = data['device_type'].apply(lambda x:0 if x< 85 else (1 if x<170 else 2))
    
    data['act_continue_std_div_mean'] = data['act_continue_std']/(data['act_continue_mean']+0.00001)
    data['app_continue_std_div_mean'] = data['app_continue_std']/(data['app_continue_mean']+0.00001)
    
    data['app_div_reg'] = data['app_day_count']/data['register_day']
    data['act_div_reg'] = data['act_day_count']/data['register_day']
    data['video_div_reg'] = data['video_day_count']/data['register_day']
    data['author_div_reg'] = data['act_author_count']/data['register_day']
    data['act_video_div_reg'] = data['act_video_nums']/data['register_day']
    
    data['act_last1days_div_reg'] = data['act_last1days_counts']/data['register_day']
    data['device_exp_reg'] = data['device_type']/(data['register_type']+0.00001)
    
    data['video_max_min'] = data['video_day_max']-data['video_day_min']
    data['app_max_min'] = data['app_day_max']-data['app_day_min']
    data['act_max_min'] = data['act_day_max']-data['act_day_min']
    
    data['act_app_day_exp'] = data['app_day_max']/(data['act_day_max']+0.00001)
    data['video_app_day_exp'] = data['app_day_max']/(data['video_day_max']+0.00001)
    
    data['app_day_diff_std_mean'] = data['app_day_diff_std']/(data['app_day_diff_mean']+0.00001)
    data['app_day_std_mean'] = data['app_day_std']/(data['app_day_mean']+0.00001)
    
    data['act_app_day_mul'] = data['app_day_max']+data['act_day_max']
    data['video_app_day_mul'] = data['app_day_max']+data['video_day_max']
    data['video_act_day_mul'] = data['act_day_max']+data['video_day_max']
    
    data['author_rate'] = data['act_author_count']/data['act_author_counts']
    
    data['act_author_rate'] = data['act_author_counts']/data['act_day_count']
    data['app_author_rate'] = data['act_author_counts']/data['app_day_count']
    data['video_author_rate'] = data['act_author_counts']/data['video_day_count']
    
    return data

time: 18.9 ms


In [75]:
def auc(y_true, y_pred):  
    ptas = tf.stack([binary_PTA(y_true,y_pred,k) for k in np.linspace(0, 1, 1000)],axis=0)  
    pfas = tf.stack([binary_PFA(y_true,y_pred,k) for k in np.linspace(0, 1, 1000)],axis=0)  
    pfas = tf.concat([tf.ones((1,)) ,pfas],axis=0)  
    binSizes = -(pfas[1:]-pfas[:-1])  
    s = ptas*binSizes  
    return K.sum(s, axis=0)  

# PFA, prob false alert for binary classifier  
def binary_PFA(y_true, y_pred, threshold=K.variable(value=0.5)):  
    y_pred = K.cast(y_pred >= threshold, 'float32')  
    # N = total number of negative labels  
    N = K.sum(1 - y_true)  
    # FP = total number of false alerts, alerts from the negative class labels  
    FP = K.sum(y_pred - y_pred * y_true)  
    return FP/N  

# P_TA prob true alerts for binary classifier  
def binary_PTA(y_true, y_pred, threshold=K.variable(value=0.5)):  
    y_pred = K.cast(y_pred >= threshold, 'float32')  
    # P = total number of positive labels  
    P = K.sum(y_true)  
    # TP = total number of correct alerts, alerts from the positive class labels  
    TP = K.sum(y_pred * y_true)  
    return TP/P 


time: 11.3 ms


In [102]:
def simple_keras(train_df,train_y,val_df,val_y,test_df):
    '''
    imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
    imp.fit(train_df)
    train_df = imp.transform(train_df)
    val_df = imp.transform(val_df)
    test_df = imp.transform(test_df)
    '''
    sc = StandardScaler()
    sc.fit(train_df)
    train_df = sc.transform(train_df)
    val_df = sc.transform(val_df)
    test_df = sc.transform(test_df)
    
    
    # model = Sequential()
    # model.add(Dense(64, input_shape=(train_df.shape[1],)))
    # model.add(Activation('tanh'))
    # model.add(Dropout(0.3))
    # model.add(Dense(64))
    # model.add(Activation('relu'))
    # model.add(Dropout(0.3))
    # model.add(Dense(64))
    # model.add(Activation('tanh'))
    # model.add(Dropout(0.3))
    # model.add(Dense(64))
    # model.add(Activation('linear'))
    # model.add(Dense(1)) # 这里需要和输出的维度一致
    # model.add(Activation('sigmoid'))
    
    model = get_model(train_df)
    
    # For a multi-class classification problem
    model.compile(loss='binary_crossentropy',
                  optimizer='nadam',
                  #metrics=['accuracy']
                  metrics=[auc]
                  )
    # model.load_weights("/home/kesci/bigdata/model/nn_model_2018-07-27 14:14:04.h5")
    time_date = time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time()))
    filepath = '/home/kesci/bigdata/model/10fold_mlp_param_%s.h5'%(str(time_date))
    
    checkpoint = ModelCheckpoint(filepath, monitor='val_auc', verbose=1, save_best_only=True, mode='max')
    
    model.fit(train_df,train_y,epochs=10,batch_size=1024,
                    #validation_data=[[X_test,X_test1],y_test],
                    validation_data=[val_df,val_y],
                    shuffle=True,verbose=2,
                    # callbacks = [EarlyStopping(monitor='val_auc',patience=5,mode='max'),checkpoint]
                    )

    # time_date = time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time()))
    model.save_weights("/home/kesci/bigdata/model/nn_model_%s.h5"%(str(time_date)))
    test_y = model.predict(test_df)
    y_pred = model.predict(val_df)
    

    return y_pred,test_y

time: 5.64 ms


In [245]:
def get_model_nn1():
    xin = Input(shape=(X.shape[1],))
    xin1 = Input(shape=(X1.shape[1],X1.shape[2]))
    
    x1 = Dense(32,init='he_normal')(xin)
    x1 = PReLU()(x1)
    x1 = BatchNormalization()(x1)
    x1b = x1

    x2 = LSTM(36,activation='relu',dropout=0.2,return_sequences=False)(xin1)
    x2 = BatchNormalization()(x2)
    x2b = x2

    x3 = Conv1D(20,3)(xin1)
    x3 = BatchNormalization()(x3)
    x3 = Activation('relu')(x3)
    x3 = MaxPooling1D(2)(x3)
    x3 = LSTM(10,activation='relu',dropout=0.2,return_sequences=False)(x3)
    x3 = BatchNormalization()(x3)
    x3b = x3
    
    xb = Concatenate(axis=1)([x1b,x2b,x3b])
    xb = Dense(64,init='he_normal')(xb)
    xb = PReLU()(xb)
    xb = BatchNormalization()(xb)
    xb = Dropout(0.25)(xb)
    xb = Dense(16,init='he_normal')(xb)
    xb = Dropout(0.5)(xb)
    y = Dense(1, activation='sigmoid')(xb)
    
    model = Model(inputs=[xin,xin1],outputs=y)
    return model

time: 6.4 ms


In [247]:
def get_model(X):
        model = Sequential()
        model.add(Dense(128, input_shape=(X.shape[1],),init='he_normal'))
        model.add(PReLU())
        model.add(BatchNormalization())
        model.add(Dropout(0.2))

        model.add(Dense(64,init='he_normal'))
    
        model.add(PReLU())
        model.add(BatchNormalization())
        model.add(Dropout(0.2))
        
        model.add(Dense(34,init='he_normal'))
    
        model.add(PReLU())
        model.add(BatchNormalization())
        model.add(Dropout(0.3))

        model.add(Dense(128,init='he_normal'))
        '''
        model.add(PReLU())
        model.add(BatchNormalization())
        model.add(Dropout(0.2))
     
        model.add(Dense(16,init='he_normal'))
        '''
        
        model.add(PReLU())
        model.add(BatchNormalization())
        model.add(Dropout(0.5))
        
        model.add(Dense(1)) 
        model.add(Activation('sigmoid'))
        
        return model


time: 3.84 ms


In [103]:
def model_train(t,test):
    
    t = t.fillna(0)
    # t = t.astype("float32")
    test = test.fillna(0)
    t_userid = t.pop('user_id')
    y = t.pop('label')
    col = [
        'register_day', 'register_type', 'device_type', 'device_type_register_count', 'device_type_count', 'device_type_register_rate', 'act_day_skew', 'act_day_sum', 'act_day_kurt', 'act_day_mean', 'act_day_max', 'act_day_min', 'act_day_std', 'act_day_count', 'act_day_var', 'act_last1days_counts', 'act_last2days_counts', 'act_last3days_counts', 'act_last4days_counts', 'act_last5days_counts', 'act_last6days_counts', 'act_last7days_counts', 'act_nums', 'author_nums', 'act_video_nums', 'action_type_nums', 'page_nums', 'act_continue_std', 'act_continue_mean', 'act_last_gap', 'act_page0_count', 'act_page2_count', 'act_page3_count', 'act_page1_count', 'act_page4_count', 'act_type0_count', 'act_type1_count', 'act_type3_count', 'act_type2_count', 'act_type4_count', 'act_type5_count', 'act_author_counts', 'act_author_std', 'act_author_mean', 'act_author_count', 'act_daily_max', 'single_video_max', 'every_video_type_mean', 'every_page_type_mean', 'video_day_var', 'video_day_count', 'video_day_min', 'video_day_std', 'video_day_mean', 'video_day_skew', 'video_day_sum', 'video_day_max', 'video_day_kurt', 'video_nums', 'video_last7days_counts', 'app_day_min', 'app_day_std', 'app_day_skew', 'app_day_mean', 'app_day_max', 'app_day_kurt', 'app_day_var', 'app_day_count', 'app_day_sum', 'app_day_diff_min', 'app_day_diff_kurt', 'app_day_diff_var', 'app_day_diff_sum', 'app_day_diff_std', 'app_day_diff_count', 'app_day_diff_skew', 'app_day_diff_max', 'app_day_diff_mean', 'app_nums', 'app_last_gap', 'app_continue_std', 'app_continue_mean', 'app_last1days_counts', 'app_last2days_counts', 'app_last3days_counts', 'app_last4days_counts', 'app_last5days_counts', 'app_last6days_counts', 'app_last7days_counts', 'type0_rate', 'type1_rate', 'type2_rate', 'type3_rate', 'page0_rate', 'page1_rate', 'page2_rate', 'page3_rate', 'page4_rate', 'is_have_act', 'is_have_video', 'is_have_type0', 'is_have_type1', 'is_have_type2', 'act_day_count1', 'act_day_max1', 'act_day_max2', 'every_video_rate', 'every_video_author_rate', 'act_daily_max_rate', 'act_single_video_rate', 'app_act_rate', 'app_video_type_rate', 'app_video_type_sub', 'page1_2_rate', 'app_std_mul', 'act_last1_mul', 'act_last1_std_mul', 'app_diff_rate', 'app_page1_rate', 'page1_type2_rate', 'app_std_div_mean', 'act_std_div_mean', 'video_std_div_mean', 'device_type1', 'act_continue_std_div_mean', 'app_continue_std_div_mean', 'app_div_reg', 'act_div_reg', 'video_div_reg', 'author_div_reg', 'act_video_div_reg', 'act_last1days_div_reg', 'device_exp_reg', 'video_max_min', 'app_max_min', 'act_max_min', 'act_app_day_exp', 'video_app_day_exp', 'app_day_diff_std_mean', 'app_day_std_mean', 'act_app_day_mul', 'video_app_day_mul', 'video_act_day_mul', 'author_rate', 'act_author_rate', 'app_author_rate', 'video_author_rate'
        ]
    X = t[col].values
    print(len(col))
    
    # test_label = test['label']
    test_userid = test.pop("user_id")
    test = test[col].values
    
    N = 10
    skf = StratifiedKFold(n_splits=N,shuffle=True,random_state=8888)
    
    xx_cv = []
    xx_pre = []
    i=1
    for train_in,test_in in skf.split(X,y):
        print("折数",i)
        X_train,X_test,y_train,y_test = X[train_in],X[test_in],y[train_in],y[test_in]
    
        y_pred, ty = simple_keras(X_train,y_train,X_test,y_test,test)
        xx_pre.append(ty)
        xx_cv.append(roc_auc_score(y_test,y_pred))
        print (roc_auc_score(y_test,y_pred))
        # print ("线下验证--》",roc_auc_score(test_label,ty))
        i = i+1
    
    s = 0
    for i in xx_pre:
        s = s + i
    s = s /N
    res = pd.DataFrame()
    res['user_id'] = list(test_userid.values)
    res['RST'] = list(s)
    # res['RST'] = res['RST'][:,0]
    print('xx_cv',np.mean(xx_cv))
    time_date = time.strftime('%Y-%m-%d',time.localtime(time.time()))
    res.to_csv('/home/kesci/bigdata/result/nn_30epoch_%s_%s.csv'%(str(time_date),str(np.mean(xx_cv)).split('.')[1]),sep=',',index=False,header=None)

time: 10.4 ms


In [248]:
gc.collect()
n1 = pd.read_csv("/home/kesci/input/home/kesci/fuck/fuck1_feature.csv")
n1 = get_rate(n1)

n2 = pd.read_csv("/home/kesci/input/home/kesci/fuck/fuck2_feature.csv")
n2 = get_rate(n2)

n = pd.concat([n1,n2],axis=0,ignore_index=True,join='outer')

n3 = pd.read_csv("/home/kesci/input/home/kesci/fuck/fuck3_feature.csv")
n3 = get_rate(n3)
# print(n1.columns.values.tolist())

model_train(n,n3)


147
折数 1


/opt/conda/lib/python3.5/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, kernel_initializer="he_normal", input_shape=(147,))`
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.5/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(64, kernel_initializer="he_normal")`
  
/opt/conda/lib/python3.5/site-packages/ipykernel_launcher.py:14: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(34, kernel_initializer="he_normal")`
  
/opt/conda/lib/python3.5/site-packages/ipykernel_launcher.py:20: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, kernel_initializer="he_normal")`


Train on 610419 samples, validate on 67825 samples
Epoch 1/10
 - 95s - loss: 0.4448 - auc: 0.8749 - val_loss: 0.4278 - val_auc: 0.8752
Epoch 2/10
 - 32s - loss: 0.4299 - auc: 0.8827 - val_loss: 0.4259 - val_auc: 0.8759
Epoch 3/10
 - 31s - loss: 0.4279 - auc: 0.8839 - val_loss: 0.4252 - val_auc: 0.8763
Epoch 4/10
 - 31s - loss: 0.4262 - auc: 0.8848 - val_loss: 0.4259 - val_auc: 0.8765
Epoch 5/10
 - 31s - loss: 0.4255 - auc: 0.8852 - val_loss: 0.4247 - val_auc: 0.8768
Epoch 6/10
 - 30s - loss: 0.4248 - auc: 0.8854 - val_loss: 0.4236 - val_auc: 0.8772
Epoch 7/10
 - 31s - loss: 0.4238 - auc: 0.8860 - val_loss: 0.4253 - val_auc: 0.8772
Epoch 8/10
 - 31s - loss: 0.4233 - auc: 0.8863 - val_loss: 0.4239 - val_auc: 0.8775
Epoch 9/10
 - 29s - loss: 0.4229 - auc: 0.8865 - val_loss: 0.4238 - val_auc: 0.8774
Epoch 10/10
 - 31s - loss: 0.4227 - auc: 0.8867 - val_loss: 0.4245 - val_auc: 0.8775
0.8858335748734163
折数 2
Train on 610419 samples, validate on 67825 samples
Epoch 1/10
 - 98s - loss: 0.4449 

In [35]:
lgbtest_y = pd.read_csv("/home/kesci/bigdata/result/lgb_2018-08-07_8824762704486109.csv",names=['id','res'])
lgbtest = lgbtest_y['res']
nnres2 = pd.read_csv("/home/kesci/bigdata/result/lgb_2018-08-07_8822277899293791.csv",names=['id','res'])
nnres = nnres2['res'].apply(lambda x: float(str(x).replace('[','').replace(']','')))

train_y2 = n2['label']
k=0.05
h=20
for i in range(1,h):
    test_y2 = lgbtest*k*i+nnres*k*(h-i)
    print("-----------")
    print (i*k,roc_auc_score(train_y2,test_y2))

-----------
0.05 0.8895918677741311
-----------
0.1 0.8896298705143676
-----------
0.15000000000000002 0.8896646339660661
-----------
0.2 0.88969508868271
-----------
0.25 0.889721847589855
-----------
0.30000000000000004 0.8897449320993398
-----------
0.35000000000000003 0.8897644231366921
-----------
0.4 0.8897803038248957
-----------
0.45 0.8897928110499418
-----------
0.5 0.8898014346210222
-----------
0.55 0.8898062664500888
-----------
0.6000000000000001 0.8898080117720305
-----------
0.65 0.8898063389371087
-----------
0.7000000000000001 0.8898010119475963
-----------
0.75 0.8897913729738529
-----------
0.8 0.8897785400364122
-----------
0.8500000000000001 0.8897609958634349
-----------
0.9 0.8897402129629279
-----------
0.9500000000000001 0.8897165400486498
time: 3.14 s


In [16]:
lgb_res = pd.read_csv("/home/kesci/work/lh.csv",names=['id','res1'])
nn_res = pd.read_csv("/home/kesci/bigdata/result/nn_mean.csv",names=['id','res2'])
res = pd.merge(lgb_res,nn_res,on="id",how = 'left')
r1 = res['res1']
r2 = res['res2'].apply(lambda x: float(str(x).replace('[','').replace(']','')))
# res['res2'] = res['res2'].apply(lambda x: float(str(x).replace('[','').replace(']','')))
# print(res.corr())
result = pd.DataFrame()
result['id'] = res['id']
result['res'] = r1*0.8 + r2*0.2
time_date = time.strftime('%Y-%m-%d',time.localtime(time.time()))
result.to_csv('/home/kesci/bigdata/result/lh_nn_%s.csv'%(str(time_date)),sep=',',index=False,header=None)

time: 4.14 s


In [18]:
path = "/mnt/datasets/fusai/"
reg_type = {'user_id':np.uint32,'register_day':np.uint8,'register_type':np.uint8,'device_type':np.uint8}
user_reg = pd.read_table(path+'user_register_log.txt',names=['user_id','register_day','register_type','device_type'],encoding='utf-8',dtype=reg_type,sep='\t')


time: 159 ms


In [19]:
out_user = user_reg[(user_reg["register_day"]==24)&(user_reg["register_type"]==3)&((user_reg["device_type"]==1)|(user_reg["device_type"]==223)|(user_reg["device_type"]==83))]
out_userid = out_user['user_id']
# out_userid

time: 8.83 ms


In [32]:
rhe = pd.read_csv("/home/kesci/bigdata/result/lh_nn_2018-08-10.csv",names=['user_id','res'])

time: 174 ms


In [36]:
(rhe[rhe['res']==0])['res'].value_counts()

0.0    13923
Name: res, dtype: int64

time: 6.78 ms


In [242]:
lgb_res = pd.read_csv("/home/kesci/bigdata/result/l_x_x1_c_lr.csv",names=['user_id','res1'])
nn_res = pd.read_csv("/home/kesci/bigdata/result/nn_2018-08-10_8881918220422907.csv",names=['user_id','res2'])
res = pd.merge(lgb_res,nn_res,on="user_id",how = 'left')
r1 = res['res1']
r2 = res['res2'].apply(lambda x: float(str(x).replace('[','').replace(']','')))
# res['res2'] = res['res2'].apply(lambda x: float(str(x).replace('[','').replace(']','')))
# print(res.corr())
result = pd.DataFrame()
result['user_id'] = res['user_id']
result['res'] = r1*0.6 + r2*0.4

result.to_csv('/home/kesci/bigdata/result/lh1.csv',sep=',',index=False,header=None)

0.9926294068199738
time: 9.1 s


In [230]:
cp /home/kesci/bigdata/result/nn_4.csv /home/kesci/work/

time: 363 ms


In [130]:
lgb = pd.read_csv('/home/kesci/work/lgb_2018-08-09_8893366235403233.csv',names=['user_id','lgb'])
xgb = pd.read_csv('/home/kesci/work/xgb_2018-08-08_889327200627967.csv',names=['user_id','xgb'])
ctb = pd.read_csv('/home/kesci/work/cat_2018-08-09_8891670632228583.csv',names=['user_id','ctb'])
xgb_lr = pd.read_csv('/home/kesci/bigdata/result/xgb_lr_2018-08-10.csv',names=['user_id','xgb_lr'])
nn_res = pd.read_csv('/home/kesci/bigdata/result/nn_2018-08-10_8881918220422907.csv',names=['user_id','nn'])
nn_res['nn'] = nn_res['nn'].apply(lambda x: float(str(x).replace('[','').replace(']','')))
lgb = lgb.merge(xgb,on="user_id",how='left')
lgb = lgb.merge(ctb,on="user_id",how='left')
lgb = lgb.merge(xgb_lr,on="user_id",how='left')
lgb = lgb.merge(nn_res,on="user_id",how='left')
print(lgb.corr())

          user_id       lgb       xgb       ctb    xgb_lr        nn
user_id  1.000000 -0.000199 -0.000238 -0.000273 -0.000367 -0.000535
lgb     -0.000199  1.000000  0.999421  0.998736  0.996799  0.995085
xgb     -0.000238  0.999421  1.000000  0.998598  0.996724  0.994983
ctb     -0.000273  0.998736  0.998598  1.000000  0.996329  0.995459
xgb_lr  -0.000367  0.996799  0.996724  0.996329  1.000000  0.993446
nn      -0.000535  0.995085  0.994983  0.995459  0.993446  1.000000
time: 2.58 s
